<a href="https://colab.research.google.com/github/hoky1227/bert_based-recommendation/blob/main/Transformer_Pre_processing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install pytorch-transformers

In [ ]:
import tensorflow as tf
import torch
import tensorflow_datasets as tfds

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# import numpy as np
# import pandas as pd
# import torch
from torch.utils.data import Dataset
# from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

import pandas as pd
import numpy as np
import random
import time
import datetime
import csv
import os
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/2017online retail.csv")
df.head()
# df.describe()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Customer,Payment Way,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,Skrill,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,Skrill,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,Skrill,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,Skrill,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,Credit,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49


In [ ]:
# Detail = pd.read_csv('/content/drive/MyDrive/Detail.csv')
# Detail.pop('Unnamed: 0')
# Detail.head()

In [ ]:
# detail = Detail['0'].unique()
# detail = detail.tolist()
# len(detail)

In [ ]:
# dic = {string : i for i,string in enumerate(detail)}
# print(dic)
# len(dic)

In [ ]:
print(df.isnull().sum())
df.dropna(inplace=True)
df.isnull().sum()

Customer       42970
Payment Way        0
Invoice            0
ItemCode           0
Detail          1745
Date               0
Amount             0
Price              0
dtype: int64


Customer       0
Payment Way    0
Invoice        0
ItemCode       0
Detail         0
Date           0
Amount         0
Price          0
dtype: int64

In [ ]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
df.head()

1458


,Customer,Payment Way,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,Skrill,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,Skrill,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,Skrill,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,Skrill,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,Credit,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49


In [ ]:
df.drop(['Payment Way'],axis=1, inplace=True)
df.head()

,Customer,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49


In [ ]:
df['Detail'] = df['Detail'].str.strip()
Detail = df['Detail'].unique()
for i in Detail:
    if i.isupper() == False:
        print(i)

Manual
BAG 250g SWIRLY MARBLES
Discount
Adjustment by Peter on 24/05/2010 1
BAG 125g SWIRLY MARBLES
POLYESTER FILLER PAD 65CMx65CM
Bank Charges
POLYESTER FILLER PAD 40x40cm
Adjustment by john on 26/01/2010 17
Adjustment by john on 26/01/2010 16
Adjust bad debt
FOLK ART GREETING CARD,pack/12
POLYESTER FILLER PAD 45x45cm
Adjustment by Peter on Jun 25 2010
ESSENTIAL BALM 3.5g TIN IN ENVELOPE
SET Of 6 SOLDIER SKITTLES
POLYESTER FILLER PAD 30CMx30CM
THE KING GIFT BAG 25x24x12cm
POLYESTER FILLER PAD 45x30cm
This is a test product.
POLYESTER FILLER PAD 60x40cm


In [ ]:
idx = df[df['Amount'] <= 0.].index
df.drop(idx, inplace=True)
l = ['Manual', 'This is a test product.', 'Discount', 'Adjustment by Peter on 24/05/2010 1',\
     'Bank Charges', 'Adjustment by john on 26/01/2010 17',  'Adjustment by john on 26/01/2010 16', 'Adjust bad debt', 'Adjustment by Peter on Jun 25 2010', 'POSTAGE', 'CARRIAGE']
for v in l:
    idx2 = df[df['Detail'] == v].index
    df.drop(idx2, inplace=True)
idx3 = df[df['Price'] <= 0.].index
df.drop(idx3, inplace=True)
print(df.head())
df.describe()

      Customer Invoice ItemCode  ...             Date Amount  Price
4492   34882.0  264212    84755  ...  2017/6/30 17:04     16   0.65
4493   34882.0  264212    20957  ...  2017/6/30 17:04     16   1.45
4494   34882.0  264212    22457  ...  2017/6/30 17:04     12   2.95
4496   34882.0  264212    20958  ...  2017/6/30 17:04      8   1.95
4497   34882.0  264212    22430  ...  2017/6/30 17:04      3   4.95

[5 rows x 7 columns]


,Customer,Amount,Price
count,161040.000000,161040.000000,161040.000000
mean,35334.207570,14.045293,3.051039
std,1673.922109,123.241911,3.435328
min,32346.000000,1.000000,0.001000
25%,33976.000000,2.000000,1.250000
50%,35248.000000,5.000000,1.950000
75%,36805.000000,12.000000,3.750000
max,38287.000000,19152.000000,125.000000


In [ ]:
df['Detail'] = df['Detail'].str.strip()
Detail = df['Detail'].unique()
for i in Detail:
    if i.isupper() == False:
        print(i)

BAG 250g SWIRLY MARBLES
FOLK ART GREETING CARD,pack/12
POLYESTER FILLER PAD 45x45cm
POLYESTER FILLER PAD 40x40cm
BAG 125g SWIRLY MARBLES
POLYESTER FILLER PAD 65CMx65CM
ESSENTIAL BALM 3.5g TIN IN ENVELOPE
SET Of 6 SOLDIER SKITTLES
POLYESTER FILLER PAD 30CMx30CM
THE KING GIFT BAG 25x24x12cm
POLYESTER FILLER PAD 45x30cm
POLYESTER FILLER PAD 60x40cm


In [ ]:
for i in Detail:
    if len(i) <= 10:
        print(i)

PHOTO CUBE
SOMBRERO
BINGO SET
SPACE OWL
CAT BOWL
SPOTS MUG
SPACE FROG
DAISY RING


In [ ]:
df.drop(['Amount', 'Price', 'ItemCode'],axis=1, inplace=True)
df.head()

,Customer,Invoice,Detail,Date
4492,34882.0,264212,COLOUR GLASS T-LIGHT HOLDER HANGING,2017/6/30 17:04
4493,34882.0,264212,PORCELAIN HANGING BELL SMALL,2017/6/30 17:04
4494,34882.0,264212,NATURAL SLATE HEART CHALKBOARD,2017/6/30 17:04
4496,34882.0,264212,PORCELAIN HANGING BELL LARGE,2017/6/30 17:04
4497,34882.0,264212,ENAMEL WATERING CAN CREAM,2017/6/30 17:04


In [ ]:
df['Invoice'] = df['Invoice'].apply(lambda x: str(x))

In [ ]:
def sortdf(data, column):
    data.sort_values(column, inplace=True)
    data.reset_index(inplace=True)
    data.pop('index')
    return data

In [ ]:
df = sortdf(df, ['Invoice'])
df.head()

,Customer,Invoice,Detail,Date
0,34590.0,243414,HAND OPEN SHAPE GOLD,2017/1/4 10:28
1,34590.0,243414,RETRO SPOT MUG,2017/1/4 10:28
2,34590.0,243414,RETRO SPOT SUGAR JAM BOWL,2017/1/4 10:28
3,34590.0,243414,NEW ENGLAND CERAMIC CAKE SERVER,2017/1/4 10:28
4,34590.0,243414,RETRO SPOT TRADITIONAL TEAPOT,2017/1/4 10:28


In [ ]:
Date = df['Date'].unique()
Date = sorted(Date)
print(len(Date))

Invoice = df['Invoice'].unique()
Invoice = sorted(Invoice)
len(Invoice)

7178


7557

In [ ]:
df.pop('Date')
df.head()

,Customer,Invoice,Detail
0,34590.0,243414,HAND OPEN SHAPE GOLD
1,34590.0,243414,RETRO SPOT MUG
2,34590.0,243414,RETRO SPOT SUGAR JAM BOWL
3,34590.0,243414,NEW ENGLAND CERAMIC CAKE SERVER
4,34590.0,243414,RETRO SPOT TRADITIONAL TEAPOT


In [ ]:
df1 = df.copy()
df1.pop('Detail')
df1.drop_duplicates(inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)
df1.drop_duplicates(inplace=True)
df1.reset_index(inplace=True)
df1.pop('index')
display(df1.head())
len(df1)

,Customer,Invoice
0,34590.0,243414
1,33287.0,243427
2,36550.0,243428
3,34680.0,243432
4,34709.0,243433


7557

In [ ]:
df2 = pd.DataFrame(columns=['Invoice', 'Detail'])
values = df['Invoice'].value_counts().index.sort_values()

for i, v in enumerate(values):
    df2.loc[i] = (v, '&&'.join(df[df['Invoice'] == v]['Detail']))
df2

,Invoice,Detail
0,243414,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,243427,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,243428,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,243432,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,243433,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...
7552,264208,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
7553,264209,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
7554,264210,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
7555,264211,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [ ]:
df1_ = df1.drop(['Invoice'], axis=1)
df1_

,Customer
0,34590.0
1,33287.0
2,36550.0
3,34680.0
4,34709.0
...,...
7552,33649.0
7553,38215.0
7554,38231.0
7555,33589.0


In [ ]:
df3 = pd.concat([df1_, df2], axis=1)
df3

,Customer,Invoice,Detail
0,34590.0,243414,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,33287.0,243427,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,36550.0,243428,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,34680.0,243432,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,34709.0,243433,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...,...
7552,33649.0,264208,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
7553,38215.0,264209,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
7554,38231.0,264210,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
7555,33589.0,264211,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [ ]:
token_df = df3.copy()
token_df['Detail'] = token_df['Detail'].str.replace('&&', ' [SEP] ')
sentences = token_df['Detail']
sentences = ["[CLS] " + str(sentence).lower() + " [SEP]" for sentence in sentences]
sentences = [sentence.replace('[sep]', '[SEP]') for sentence in sentences]
sentences[0]

'[CLS] hand open shape gold [SEP] retro spot mug [SEP] retro spot sugar jam bowl [SEP] new england ceramic cake server [SEP] retro spot traditional teapot [SEP] retro spot large milk jug [SEP]'

In [ ]:
token_df.head()

,Customer,Invoice,Detail
0,34590.0,243414,HAND OPEN SHAPE GOLD [SEP] RETRO SPOT MUG [SEP...
1,33287.0,243427,MOROCCAN TEA GLASS [SEP] BLACK FLOWER CANDLE P...
2,36550.0,243428,LUNCHBOX WITH CUTLERY FAIRY CAKES [SEP] STRAWB...
3,34680.0,243432,RED WOOLLY HOTTIE WHITE HEART. [SEP] CERAMIC C...
4,34709.0,243433,STRAWBERRY CERAMIC TRINKET BOX [SEP] JUMBO BA...


In [ ]:
def BERTtokenizer(sentences):
    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    # tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    return tokenized_texts

In [ ]:
tokenized_texts = BERTtokenizer(sentences)

print (sentences[0])
print (tokenized_texts[0])
len(tokenized_texts)

[CLS] hand open shape gold [SEP] retro spot mug [SEP] retro spot sugar jam bowl [SEP] new england ceramic cake server [SEP] retro spot traditional teapot [SEP] retro spot large milk jug [SEP]
['[CLS]', 'hand', 'open', 'shape', 'gold', '[SEP]', 'retro', 'spot', 'mug', '[SEP]', 'retro', 'spot', 'sugar', 'jam', 'bowl', '[SEP]', 'new', 'england', 'ceramic', 'cake', 'server', '[SEP]', 'retro', 'spot', 'traditional', 'tea', '##pot', '[SEP]', 'retro', 'spot', 'large', 'milk', 'jug', '[SEP]']


7557

In [ ]:
def SPACEtokenizer(sentences):
    tokenized_texts2 = [sent.split(' ') for sent in sentences]
    return tokenized_texts2

In [ ]:
tokenized_texts2 = SPACEtokenizer(sentences)
print(tokenized_texts2[0])
len(tokenized_texts2)

['[CLS]', 'hand', 'open', 'shape', 'gold', '[SEP]', 'retro', 'spot', 'mug', '[SEP]', 'retro', 'spot', 'sugar', 'jam', 'bowl', '[SEP]', 'new', 'england', 'ceramic', 'cake', 'server', '[SEP]', 'retro', 'spot', 'traditional', 'teapot', '[SEP]', 'retro', 'spot', 'large', 'milk', 'jug', '[SEP]']


7557

In [ ]:
tokenized_df = token_df.copy()
tokenized_df['Detail'] = tokenized_texts
# print(len(tokenized_df))
tokenized_df

,Customer,Invoice,Detail
0,34590.0,243414,"[[CLS], hand, open, shape, gold, [SEP], retro,..."
1,33287.0,243427,"[[CLS], moroccan, tea, glass, [SEP], black, fl..."
2,36550.0,243428,"[[CLS], lunch, ##box, with, cutler, ##y, fairy..."
3,34680.0,243432,"[[CLS], red, wool, ##ly, hot, ##tie, white, he..."
4,34709.0,243433,"[[CLS], strawberry, ceramic, tri, ##nk, ##et, ..."
...,...,...,...
7552,33649.0,264208,"[[CLS], ass, ##orted, colour, bird, or, ##name..."
7553,38215.0,264209,"[[CLS], pack, of, 60, pink, paisley, cake, cas..."
7554,38231.0,264210,"[[CLS], glass, jar, digest, ##ive, biscuits, [..."
7555,33589.0,264211,"[[CLS], picnic, basket, wi, ##cker, small, [SE..."


In [ ]:
tokenized_df['Detail'] = tokenized_df['Detail'].apply(lambda x: ','.join(map(str, x)))
tokenized_df.head()

,Customer,Invoice,Detail
0,34590.0,243414,"[CLS],hand,open,shape,gold,[SEP],retro,spot,mu..."
1,33287.0,243427,"[CLS],moroccan,tea,glass,[SEP],black,flower,ca..."
2,36550.0,243428,"[CLS],lunch,##box,with,cutler,##y,fairy,cakes,..."
3,34680.0,243432,"[CLS],red,wool,##ly,hot,##tie,white,heart,.,[S..."
4,34709.0,243433,"[CLS],strawberry,ceramic,tri,##nk,##et,box,[SE..."


In [ ]:
tokenized_df2 = token_df.copy()
tokenized_df2['Detail'] = tokenized_texts2
# print(len(tokenized_df2))
tokenized_df2

,Customer,Invoice,Detail
0,34590.0,243414,"[[CLS], hand, open, shape, gold, [SEP], retro,..."
1,33287.0,243427,"[[CLS], moroccan, tea, glass, [SEP], black, fl..."
2,36550.0,243428,"[[CLS], lunchbox, with, cutlery, fairy, cakes,..."
3,34680.0,243432,"[[CLS], red, woolly, hottie, white, heart., [S..."
4,34709.0,243433,"[[CLS], strawberry, ceramic, trinket, box, [SE..."
...,...,...,...
7552,33649.0,264208,"[[CLS], assorted, colour, bird, ornament, [SEP..."
7553,38215.0,264209,"[[CLS], pack, of, 60, pink, paisley, cake, cas..."
7554,38231.0,264210,"[[CLS], glass, jar, digestive, biscuits, [SEP]..."
7555,33589.0,264211,"[[CLS], picnic, basket, wicker, small, [SEP], ..."


In [ ]:
tokenized_df2['Detail'] = tokenized_df2['Detail'].apply(lambda x: ','.join(map(str, x)))
tokenized_df2.head()

,Customer,Invoice,Detail
0,34590.0,243414,"[CLS],hand,open,shape,gold,[SEP],retro,spot,mu..."
1,33287.0,243427,"[CLS],moroccan,tea,glass,[SEP],black,flower,ca..."
2,36550.0,243428,"[CLS],lunchbox,with,cutlery,fairy,cakes,[SEP],..."
3,34680.0,243432,"[CLS],red,woolly,hottie,white,heart.,[SEP],cer..."
4,34709.0,243433,"[CLS],strawberry,ceramic,trinket,box,[SEP],jum..."


In [ ]:
Customer = df3['Customer'].unique()
len(Customer)

2732

In [ ]:
MAX_LEN = 512

In [ ]:
overlen = []
tokenized_texts_copy = tokenized_texts.copy()
for i in range(len(tokenized_texts_copy)):
    if len(tokenized_texts_copy[i]) > MAX_LEN:
        overlen.append(tokenized_texts_copy[i])
        while 1:
            id = tokenized_texts_copy[i].index('[SEP]')
            tokenized_texts_copy[i] = tokenized_texts_copy[i][id:]
            tokenized_texts_copy[i][0] = '[CLS]'
            if len(tokenized_texts_copy[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts_copy)):
    if len(tokenized_texts_copy[i]) > MAX_LEN:
        tok.append(len(tokenized_texts_copy[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts_copy))
print(len(overlen[0]))
print(len(overlen))
print(overlen[0])

0
7557
694
185
['[CLS]', 'set', 'of', '36', 'dinosaur', 'paper', 'doi', '##lies', '[SEP]', 'candy', 'heart', 'hanging', 'decoration', '[SEP]', 'lavender', 'scented', 'set', '/', '9', 't', '-', 'lights', '[SEP]', 'white', 'hanging', 'heart', 't', '-', 'light', 'holder', '[SEP]', '72', 'sweetheart', 'fairy', 'cake', 'cases', '[SEP]', 'red', 'wool', '##ly', 'hot', '##tie', 'white', 'heart', '.', '[SEP]', 'red', 'hanging', 'heart', 't', '-', 'light', 'holder', '[SEP]', 'food', 'container', 'set', '3', 'love', 'heart', '[SEP]', 'ladies', '&', 'gentlemen', 'metal', 'sign', '[SEP]', 'ci', '##nam', '##mon', 'set', 'of', '9', 't', '-', 'lights', '[SEP]', 'please', 'one', 'person', 'metal', 'sign', '[SEP]', 'red', 'spot', 'heart', 'hot', 'water', 'bottle', '[SEP]', 'small', 'glass', 'heart', 'tri', '##nk', '##et', 'pot', '[SEP]', 'mushroom', 'blue', 'hot', 'water', 'bottle', '[SEP]', 'pack', '20', 'dolly', 'peg', '##s', '[SEP]', 'jungle', 'pops', '##icles', 'ice', 'lo', '##lly', 'holders', '[SEP

In [ ]:
overlen2 = []
tokenized_texts_copy2 = tokenized_texts2.copy()
for i in range(len(tokenized_texts_copy2)):
    if len(tokenized_texts_copy2[i]) > MAX_LEN:
        overlen2.append(tokenized_texts_copy2[i])
        while 1:
            id = tokenized_texts_copy2[i].index('[SEP]')
            tokenized_texts_copy2[i] = tokenized_texts_copy2[i][id:]
            tokenized_texts_copy2[i][0] = '[CLS]'
            if len(tokenized_texts_copy2[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts_copy2)):
    if len(tokenized_texts_copy2[i]) > MAX_LEN:
        tok.append(len(tokenized_texts_copy2[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts_copy2))
print(len(overlen2[0]))
print(len(overlen2))
print(overlen2[0])

0
7557
577
115
['[CLS]', 'set', 'of', '36', 'dinosaur', 'paper', 'doilies', '[SEP]', 'candy', 'heart', 'hanging', 'decoration', '[SEP]', 'lavender', 'scented', 'set/9', 't-lights', '[SEP]', 'white', 'hanging', 'heart', 't-light', 'holder', '[SEP]', '72', 'sweetheart', 'fairy', 'cake', 'cases', '[SEP]', 'red', 'woolly', 'hottie', 'white', 'heart.', '[SEP]', 'red', 'hanging', 'heart', 't-light', 'holder', '[SEP]', 'food', 'container', 'set', '3', 'love', 'heart', '[SEP]', 'ladies', '&', 'gentlemen', 'metal', 'sign', '[SEP]', 'cinammon', 'set', 'of', '9', 't-lights', '[SEP]', 'please', 'one', 'person', '', 'metal', 'sign', '[SEP]', 'red', 'spot', 'heart', 'hot', 'water', 'bottle', '[SEP]', 'small', 'glass', 'heart', 'trinket', 'pot', '[SEP]', 'mushroom', 'blue', 'hot', 'water', 'bottle', '[SEP]', 'pack', '20', 'dolly', 'pegs', '[SEP]', 'jungle', 'popsicles', 'ice', 'lolly', 'holders', '[SEP]', 'polyester', 'filler', 'pad', '30cmx30cm', '[SEP]', 'set', 'of', '72', 'pink', 'heart', 'paper',

In [ ]:
idx = []
for i in range(len(overlen)):
    idx.append(tokenized_df.index[tokenized_df['Detail'] == ','.join(v for v in overlen[i])].tolist())
print(len(idx))
print(idx)

185
[[59], [76], [83], [144], [245], [273], [309], [315], [377], [398], [456], [508], [530], [594], [603], [624], [626], [676], [707], [727], [901], [927], [928], [1010], [1044], [1095], [1096], [1177], [1242], [1428], [1431], [1440], [1461], [1519], [1526], [1579], [1627], [1643], [1679], [1725], [1790], [1886], [1934], [1975], [1992], [1999], [2083], [2144], [2147], [2151], [2175], [2217], [2292], [2310], [2324], [2337], [2340], [2470], [2542], [2553], [2620], [2658], [2852], [2887], [2933], [2969], [3001], [3079], [3138], [3155], [3163], [3221], [3270], [3274], [3291], [3323], [3442], [3605], [3617], [3716], [3762], [3780], [3834], [3871], [3910], [3931], [4052], [4073], [4080], [4085], [4193], [4203], [4404], [4430], [4478], [4577], [4596], [4677], [4706], [4785], [4792], [4832], [4937], [4962], [4976], [4990], [5023], [5047], [5058], [5061], [5063], [5169], [5172], [5199], [5229], [5404], [5414], [5415], [5423], [5434], [5479], [5480], [5528], [5564], [5612], [5703], [5728], [5733

In [ ]:
idx2 = []
for i in range(len(overlen2)):
    idx2.append(tokenized_df2.index[tokenized_df2['Detail'] == ','.join(v for v in overlen2[i])].tolist())
print(len(idx2))
print(idx2)

115
[[59], [76], [83], [144], [245], [273], [309], [315], [377], [398], [456], [508], [594], [624], [676], [707], [727], [901], [927], [1010], [1095], [1096], [1177], [1242], [1428], [1440], [1519], [1579], [1627], [1679], [1790], [1886], [1975], [1992], [2083], [2144], [2147], [2151], [2175], [2292], [2324], [2337], [2340], [2470], [2553], [2658], [2933], [2969], [3079], [3155], [3221], [3274], [3323], [3617], [3871], [3910], [3931], [4085], [4193], [4203], [4404], [4430], [4577], [4677], [4706], [4792], [4832], [4976], [4990], [5023], [5047], [5058], [5061], [5063], [5169], [5404], [5414], [5415], [5434], [5479], [5528], [5612], [5703], [5728], [5852], [6097], [6173], [6210], [6282], [6283], [6300], [6383], [6384], [6410], [6426], [6478], [6496], [6534], [6711], [6747], [6774], [6796], [6797], [6826], [6892], [7016], [7073], [7091], [7097], [7143], [7321], [7331], [7386], [7467], [7544]]


In [ ]:
overlen_df = pd.DataFrame(columns=df3.columns)
df_reduced = df3.copy()
for i, v in enumerate(idx):
    overlen_df.loc[i] = tokenized_df.loc[v[0]]
    df_reduced.drop(v[0], inplace=True)
overlen_df

,Customer,Invoice,Detail
0,37965.0,243759,"[CLS],set,of,36,dinosaur,paper,doi,##lies,[SEP..."
1,33081.0,243812,"[CLS],s,/,4,black,dinner,candle,silver,fl,##oc..."
2,34156.0,243819,"[CLS],scandinavian,paisley,picnic,bag,[SEP],pa..."
3,32714.0,243964,"[CLS],deco,##up,##age,,,greeting,card,,,[SEP],..."
4,34081.0,244239,"[CLS],kings,choice,mug,[SEP],white,spot,blue,c..."
...,...,...,...
180,34049.0,263952,"[CLS],fairy,soap,soap,holder,[SEP],ceramic,hea..."
181,34426.0,263953,"[CLS],kings,choice,mug,[SEP],home,sweet,home,m..."
182,35532.0,263972,"[CLS],12,ivory,rose,peg,place,settings,[SEP],l..."
183,32415.0,263987,"[CLS],paper,bun,##ting,retro,spots,[SEP],paper..."


In [ ]:
overlen_df2 = pd.DataFrame(columns=df3.columns)
df_reduced2 = df3.copy()
for i, v in enumerate(idx2):
    overlen_df2.loc[i] = tokenized_df2.loc[v[0]]
    df_reduced2.drop(v[0], inplace=True)
overlen_df2

,Customer,Invoice,Detail
0,37965.0,243759,"[CLS],set,of,36,dinosaur,paper,doilies,[SEP],c..."
1,33081.0,243812,"[CLS],s/4,black,dinner,candle,silver,floc,[SEP..."
2,34156.0,243819,"[CLS],scandinavian,paisley,picnic,bag,[SEP],pa..."
3,32714.0,243964,"[CLS],decoupage,greeting,card,,[SEP],elephant,..."
4,34081.0,244239,"[CLS],kings,choice,mug,[SEP],white,spot,blue,c..."
...,...,...,...
110,35998.0,263615,"[CLS],retro,spot,,cigar,box,matches,[SEP],gree..."
111,34505.0,263625,"[CLS],paper,bunting,white,lace,[SEP],suki,,sho..."
112,32451.0,263762,"[CLS],red,spotty,shopping,bag,[SEP],red,spotty..."
113,34049.0,263952,"[CLS],fairy,soap,soap,holder,[SEP],ceramic,hea..."


In [ ]:
df_reduced

,Customer,Invoice,Detail
0,34590.0,243414,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,33287.0,243427,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,36550.0,243428,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,34680.0,243432,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,34709.0,243433,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...,...
7552,33649.0,264208,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
7553,38215.0,264209,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
7554,38231.0,264210,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
7555,33589.0,264211,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [ ]:
df_reduced2

,Customer,Invoice,Detail
0,34590.0,243414,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,33287.0,243427,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,36550.0,243428,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,34680.0,243432,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,34709.0,243433,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...,...
7552,33649.0,264208,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
7553,38215.0,264209,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
7554,38231.0,264210,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
7555,33589.0,264211,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [ ]:
df_reduced = sortdf(df_reduced, ['Customer', 'Invoice'])
df_reduced

,Customer,Invoice,Detail
0,32346.0,249763,EDWARDIAN PARASOL PINK&&EDWARDIAN PARASOL BLAC...
1,32346.0,263774,DOORMAT 3 SMILEY CATS&&DOORMAT WELCOME SUNRISE...
2,32349.0,256394,SET OF 6 HEART CHOPSTICKS&&SPOTS MUG&&PINK DOU...
3,32355.0,259300,TOOTHPASTE TUBE PEN&&KIDS RAIN MAC BLUE&&KIDS ...
4,32358.0,261229,ASSORTED ICE CREAM FRIDGE MAGNETS&&ASSORTED SA...
...,...,...,...
7367,38283.0,248547,FUNKY MONKEY MUG&&PINK BLUE FELT CRAFT TRINKET...
7368,38283.0,249771,PACK 20 ENGLISH ROSE PAPER NAPKINS&&60 TEATIME...
7369,38283.0,252841,ASSORTED ICE CREAM FRIDGE MAGNETS&&BIG DOUGHNU...
7370,38285.0,248149,VICTORIAN GLASS HANGING T-LIGHT&&PAPER BUNTING...


In [ ]:
df_reduced2 = sortdf(df_reduced2, ['Customer', 'Invoice'])
df_reduced2

,Customer,Invoice,Detail
0,32346.0,249763,EDWARDIAN PARASOL PINK&&EDWARDIAN PARASOL BLAC...
1,32346.0,263774,DOORMAT 3 SMILEY CATS&&DOORMAT WELCOME SUNRISE...
2,32349.0,256394,SET OF 6 HEART CHOPSTICKS&&SPOTS MUG&&PINK DOU...
3,32355.0,259300,TOOTHPASTE TUBE PEN&&KIDS RAIN MAC BLUE&&KIDS ...
4,32358.0,261229,ASSORTED ICE CREAM FRIDGE MAGNETS&&ASSORTED SA...
...,...,...,...
7437,38283.0,248547,FUNKY MONKEY MUG&&PINK BLUE FELT CRAFT TRINKET...
7438,38283.0,249771,PACK 20 ENGLISH ROSE PAPER NAPKINS&&60 TEATIME...
7439,38283.0,252841,ASSORTED ICE CREAM FRIDGE MAGNETS&&BIG DOUGHNU...
7440,38285.0,248149,VICTORIAN GLASS HANGING T-LIGHT&&PAPER BUNTING...


In [ ]:
df_add = pd.DataFrame(columns=['Customer', 'Detail'])

for i in range(len(df_reduced) - 1):
    if df_reduced.loc[i, 'Customer'] != df_reduced.loc[i+1, 'Customer']:
        continue
    df_add.loc[i] = (df_reduced.loc[i, 'Customer'], df_reduced.loc[i, 'Detail'] + df_reduced.loc[i+1, 'Detail'])
df_add

,Customer,Detail
0,32346.0,EDWARDIAN PARASOL PINK&&EDWARDIAN PARASOL BLAC...
5,32359.0,TEA BAG PLATE RED SPOTTY&&FAIRY SOAP SOAP HOLD...
6,32359.0,RED GINGHAM TEDDY BEAR&&PIN CUSHION RUSSIAN DO...
8,32360.0,ASSORTED CAKES FRIDGE MAGNETS&&BUNNY DECORATIO...
10,32361.0,LUNCH BAG RED SPOTTY&&STRAWBERRY LUNCHBOX WITH...
...,...,...
7357,38268.0,DOOR MAT HEARTS&&ENAMEL PINK TEA CONTAINER&&EN...
7360,38272.0,KINGS CHOICE TEA CADDY&&RETRO RED SPOTTY WASHI...
7361,38272.0,"PORCELAIN ROSE SMALL&&KEY FOB , GARAGE DESIGN&..."
7367,38283.0,FUNKY MONKEY MUG&&PINK BLUE FELT CRAFT TRINKET...


In [ ]:
df_add2 = pd.DataFrame(columns=['Customer', 'Detail'])

for i in range(len(df_reduced2) - 1):
    if df_reduced2.loc[i, 'Customer'] != df_reduced2.loc[i+1, 'Customer']:
        continue
    df_add2.loc[i] = (df_reduced2.loc[i, 'Customer'], df_reduced2.loc[i, 'Detail'] + df_reduced2.loc[i+1, 'Detail'])
df_add2

,Customer,Detail
0,32346.0,EDWARDIAN PARASOL PINK&&EDWARDIAN PARASOL BLAC...
5,32359.0,TEA BAG PLATE RED SPOTTY&&FAIRY SOAP SOAP HOLD...
6,32359.0,RED GINGHAM TEDDY BEAR&&PIN CUSHION RUSSIAN DO...
8,32360.0,ASSORTED CAKES FRIDGE MAGNETS&&BUNNY DECORATIO...
10,32361.0,LUNCH BAG RED SPOTTY&&STRAWBERRY LUNCHBOX WITH...
...,...,...
7427,38268.0,DOOR MAT HEARTS&&ENAMEL PINK TEA CONTAINER&&EN...
7430,38272.0,KINGS CHOICE TEA CADDY&&RETRO RED SPOTTY WASHI...
7431,38272.0,"PORCELAIN ROSE SMALL&&KEY FOB , GARAGE DESIGN&..."
7437,38283.0,FUNKY MONKEY MUG&&PINK BLUE FELT CRAFT TRINKET...


In [ ]:
data_bert = pd.concat([df3.drop('Invoice', axis=1), df_add])
data_bert.reset_index(inplace=True)
data_bert.pop('index')
data_bert

,Customer,Detail
0,34590.0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,33287.0,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,36550.0,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,34680.0,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,34709.0,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...
12233,38268.0,DOOR MAT HEARTS&&ENAMEL PINK TEA CONTAINER&&EN...
12234,38272.0,KINGS CHOICE TEA CADDY&&RETRO RED SPOTTY WASHI...
12235,38272.0,"PORCELAIN ROSE SMALL&&KEY FOB , GARAGE DESIGN&..."
12236,38283.0,FUNKY MONKEY MUG&&PINK BLUE FELT CRAFT TRINKET...


In [ ]:
data_space = pd.concat([df3.drop('Invoice', axis=1), df_add2])
data_space.reset_index(inplace=True)
data_space.pop('index')
data_space

,Customer,Detail
0,34590.0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,33287.0,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,36550.0,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,34680.0,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,34709.0,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...
12286,38268.0,DOOR MAT HEARTS&&ENAMEL PINK TEA CONTAINER&&EN...
12287,38272.0,KINGS CHOICE TEA CADDY&&RETRO RED SPOTTY WASHI...
12288,38272.0,"PORCELAIN ROSE SMALL&&KEY FOB , GARAGE DESIGN&..."
12289,38283.0,FUNKY MONKEY MUG&&PINK BLUE FELT CRAFT TRINKET...


In [ ]:
# temp_bert = data_bert['Detail'].str.replace(' [SEP] ', '*')
# type(temp_bert)
series_bert = data_bert['Detail'].str.split('&&')
print(type(series_bert))
display(series_bert)
print(series_bert[0][-1])
len(series_bert[0])

<class 'pandas.core.series.Series'>


0        [HAND OPEN SHAPE GOLD, RETRO SPOT MUG, RETRO S...
1        [MOROCCAN TEA GLASS, BLACK FLOWER CANDLE PLATE...
2        [LUNCHBOX WITH CUTLERY FAIRY CAKES, STRAWBERRY...
3        [RED WOOLLY HOTTIE WHITE HEART., CERAMIC CAKE ...
4        [STRAWBERRY CERAMIC TRINKET BOX, JUMBO  BAG BA...
                               ...                        
12233    [DOOR MAT HEARTS, ENAMEL PINK TEA CONTAINER, E...
12234    [KINGS CHOICE TEA CADDY, RETRO RED SPOTTY WASH...
12235    [PORCELAIN ROSE SMALL, KEY FOB , GARAGE DESIGN...
12236    [FUNKY MONKEY MUG, PINK BLUE FELT CRAFT TRINKE...
12237    [PACK 20 ENGLISH ROSE PAPER NAPKINS, 60 TEATIM...
Name: Detail, Length: 12238, dtype: object

RETRO SPOT LARGE MILK JUG


6

In [ ]:
series_space = data_space['Detail'].str.split('&&')
print(type(series_space))
print(len(series_space))
print(series_space[0][-1])
len(series_space[0])

<class 'pandas.core.series.Series'>
12291
RETRO SPOT LARGE MILK JUG


6

In [ ]:
label_bert = []

for i in range(len(series_bert)):
    l = series_bert[i][-1]
    series_bert[i].pop()
    label_bert.append(l)

print(label_bert[:5])
# series_bert[0][-1]
len(label_bert)

['RETRO SPOT LARGE MILK JUG', 'WHITE ROUND PORCELAIN TLIGHT HOLDER', 'FLAMINGO LIGHTS', 'WHITE TISSUE REAM', 'TEA TIME MUG IN GIFT BOX']


12238

In [ ]:
label_space = []

for i in range(len(series_space)):
    l = series_space[i][-1]
    series_space[i].pop()
    label_space.append(l)

print(label_space[:5])
len(label_space)

['RETRO SPOT LARGE MILK JUG', 'WHITE ROUND PORCELAIN TLIGHT HOLDER', 'FLAMINGO LIGHTS', 'WHITE TISSUE REAM', 'TEA TIME MUG IN GIFT BOX']


12291

In [ ]:
data_bert['Detail'] = series_bert.apply(lambda x: '&&'.join(map(str, x)))
data_bert.head()

,Customer,Detail
0,34590.0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,33287.0,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,36550.0,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,34680.0,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,34709.0,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...


In [ ]:
data_space['Detail'] = series_space.apply(lambda x: '&&'.join(map(str, x)))
data_space.head()

,Customer,Detail
0,34590.0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,33287.0,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,36550.0,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,34680.0,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,34709.0,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...


In [ ]:
data_bert['label'] = label_bert
data_space['label'] = label_space
data_bert.dropna()
data_space.dropna()

,Customer,Detail,label
0,34590.0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...,RETRO SPOT LARGE MILK JUG
1,33287.0,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...,WHITE ROUND PORCELAIN TLIGHT HOLDER
2,36550.0,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...,FLAMINGO LIGHTS
3,34680.0,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...,WHITE TISSUE REAM
4,34709.0,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...,TEA TIME MUG IN GIFT BOX
...,...,...,...
12286,38268.0,DOOR MAT HEARTS&&ENAMEL PINK TEA CONTAINER&&EN...,PINK SPOTTY BOWL
12287,38272.0,KINGS CHOICE TEA CADDY&&RETRO RED SPOTTY WASHI...,PINK HEART SHAPE LOVE BUCKET
12288,38272.0,"PORCELAIN ROSE SMALL&&KEY FOB , GARAGE DESIGN&...",CERAMIC PIRATE CHEST MONEY BANK
12289,38283.0,FUNKY MONKEY MUG&&PINK BLUE FELT CRAFT TRINKET...,ROUND SNACK BOXES SET OF 4 SKULLS


In [ ]:
print(sorted(data_bert['Detail'].unique())[0])
sorted(data_space['Detail'].unique())[0]

''

In [ ]:
idx = data_bert[data_bert['Detail'] == ''].index
data_bert.drop(idx, inplace=True)
idx = data_space[data_space['Detail'] == ''].index
data_space.drop(idx, inplace=True)

In [ ]:
data_bert.reset_index(inplace=True)
data_bert.pop('index')
data_space.reset_index(inplace=True)
data_space.pop('index')

0            0
1            1
2            2
3            3
4            4
         ...  
11623    12286
11624    12287
11625    12288
11626    12289
11627    12290
Name: index, Length: 11628, dtype: int64

In [ ]:
token_data_bert = data_bert.copy()
token_data_bert['Detail'] = token_data_bert['Detail'].str.replace('&&', ' [SEP] ')
sentences_bert = token_data_bert['Detail']
sentences_bert = ["[CLS] " + str(sentence).lower() + " [SEP]" for sentence in sentences_bert]
sentences_bert = [sentence.replace('[sep]', '[SEP]') for sentence in sentences_bert]
sentences_bert[0]

'[CLS] hand open shape gold [SEP] retro spot mug [SEP] retro spot sugar jam bowl [SEP] new england ceramic cake server [SEP] retro spot traditional teapot [SEP]'

In [ ]:
token_data_space = data_space.copy()
token_data_space['Detail'] = token_data_space['Detail'].str.replace('&&', ' [SEP] ')
sentences_space = token_data_space['Detail']
sentences_space = ["[CLS] " + str(sentence).lower() + " [SEP]" for sentence in sentences_space]
sentences_space = [sentence.replace('[sep]', '[SEP]') for sentence in sentences_space]
sentences_space[0]

'[CLS] hand open shape gold [SEP] retro spot mug [SEP] retro spot sugar jam bowl [SEP] new england ceramic cake server [SEP] retro spot traditional teapot [SEP]'

In [ ]:
tokenized_texts_bert = BERTtokenizer(sentences_bert)

print (sentences_bert[0])
print (tokenized_texts_bert[0])
len(tokenized_texts_bert)

[CLS] hand open shape gold [SEP] retro spot mug [SEP] retro spot sugar jam bowl [SEP] new england ceramic cake server [SEP] retro spot traditional teapot [SEP]
['[CLS]', 'hand', 'open', 'shape', 'gold', '[SEP]', 'retro', 'spot', 'mug', '[SEP]', 'retro', 'spot', 'sugar', 'jam', 'bowl', '[SEP]', 'new', 'england', 'ceramic', 'cake', 'server', '[SEP]', 'retro', 'spot', 'traditional', 'tea', '##pot', '[SEP]']


12238

In [ ]:
tokenized_texts_space = SPACEtokenizer(sentences_space)
print(tokenized_texts_space[0])
len(tokenized_texts_space)

['[CLS]', 'hand', 'open', 'shape', 'gold', '[SEP]', 'retro', 'spot', 'mug', '[SEP]', 'retro', 'spot', 'sugar', 'jam', 'bowl', '[SEP]', 'new', 'england', 'ceramic', 'cake', 'server', '[SEP]', 'retro', 'spot', 'traditional', 'teapot', '[SEP]']


12291

자르기

In [ ]:
# overlen = []
tokenized_texts_bert_copy = tokenized_texts_bert.copy()
for i in range(len(tokenized_texts_bert_copy)):
    if len(tokenized_texts_bert_copy[i]) > MAX_LEN:
        # overlen.append(tokenized_texts_bert_copy[i])
        while 1:
            id = tokenized_texts_bert_copy[i].index('[SEP]')
            tokenized_texts_bert_copy[i] = tokenized_texts_bert_copy[i][id:]
            tokenized_texts_bert_copy[i][0] = '[CLS]'
            if len(tokenized_texts_bert_copy[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts_bert_copy)):
    if len(tokenized_texts_bert_copy[i]) > MAX_LEN:
        tok.append(len(tokenized_texts_bert_copy[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts_bert_copy))
# print(len(overlen[0]))
# print(len(overlen))
# print(overlen[0])

0
12238


In [ ]:
# overlen = []
tokenized_texts_space_copy = tokenized_texts_space.copy()
for i in range(len(tokenized_texts_space_copy)):
    if len(tokenized_texts_space_copy[i]) > MAX_LEN:
        # overlen.append(tokenized_texts_space_copy[i])
        while 1:
            id = tokenized_texts_space_copy[i].index('[SEP]')
            tokenized_texts_space_copy[i] = tokenized_texts_space_copy[i][id:]
            tokenized_texts_space_copy[i][0] = '[CLS]'
            if len(tokenized_texts_space_copy[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts_space_copy)):
    if len(tokenized_texts_space_copy[i]) > MAX_LEN:
        tok.append(len(tokenized_texts_space_copy[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts_space_copy))
# print(len(overlen[0]))
# print(len(overlen))
# print(overlen[0])

0
12291


In [ ]:
token_data_bert['Detail'] = tokenized_texts_bert_copy
token_data_space['Detail'] = tokenized_texts_space_copy

In [ ]:
token_data_bert['Detail'] = token_data_bert['Detail'].apply(lambda x: ','.join(map(str, x)))
token_data_space['Detail'] = token_data_space['Detail'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
idx = token_data_bert[token_data_bert['Detail'] == '[CLS],[SEP]'].index
token_data_bert.drop(idx, inplace=True)
idx = token_data_space[token_data_space['Detail'] == '[CLS],,[SEP]'].index
token_data_space.drop(idx, inplace=True)

In [ ]:
token_data_bert.reset_index(inplace=True)
token_data_bert.pop('index')
token_data_space.reset_index(inplace=True)
token_data_space.pop('index')

0            0
1            1
2            2
3            3
4            4
         ...  
11623    12286
11624    12287
11625    12288
11626    12289
11627    12290
Name: index, Length: 11628, dtype: int64

In [ ]:
token_data_bert

,Customer,Detail,label
0,34590.0,"[CLS],hand,open,shape,gold,[SEP],retro,spot,mu...",RETRO SPOT LARGE MILK JUG
1,33287.0,"[CLS],moroccan,tea,glass,[SEP],black,flower,ca...",WHITE ROUND PORCELAIN TLIGHT HOLDER
2,36550.0,"[CLS],lunch,##box,with,cutler,##y,fairy,cakes,...",FLAMINGO LIGHTS
3,34680.0,"[CLS],red,wool,##ly,hot,##tie,white,heart,.,[S...",WHITE TISSUE REAM
4,34709.0,"[CLS],strawberry,ceramic,tri,##nk,##et,box,[SE...",TEA TIME MUG IN GIFT BOX
...,...,...,...
11570,38268.0,"[CLS],door,mat,hearts,[SEP],enamel,pink,tea,co...",PINK SPOTTY BOWL
11571,38272.0,"[CLS],kings,choice,tea,cad,##dy,[SEP],retro,re...",PINK HEART SHAPE LOVE BUCKET
11572,38272.0,"[CLS],porcelain,rose,small,[SEP],key,f,##ob,,,...",CERAMIC PIRATE CHEST MONEY BANK
11573,38283.0,"[CLS],funky,monkey,mug,[SEP],pink,blue,felt,cr...",ROUND SNACK BOXES SET OF 4 SKULLS


In [ ]:
token_data_space

,Customer,Detail,label
0,34590.0,"[CLS],hand,open,shape,gold,[SEP],retro,spot,mu...",RETRO SPOT LARGE MILK JUG
1,33287.0,"[CLS],moroccan,tea,glass,[SEP],black,flower,ca...",WHITE ROUND PORCELAIN TLIGHT HOLDER
2,36550.0,"[CLS],lunchbox,with,cutlery,fairy,cakes,[SEP],...",FLAMINGO LIGHTS
3,34680.0,"[CLS],red,woolly,hottie,white,heart.,[SEP],cer...",WHITE TISSUE REAM
4,34709.0,"[CLS],strawberry,ceramic,trinket,box,[SEP],jum...",TEA TIME MUG IN GIFT BOX
...,...,...,...
11623,38268.0,"[CLS],door,mat,hearts,[SEP],enamel,pink,tea,co...",PINK SPOTTY BOWL
11624,38272.0,"[CLS],kings,choice,tea,caddy,[SEP],retro,red,s...",PINK HEART SHAPE LOVE BUCKET
11625,38272.0,"[CLS],porcelain,rose,small,[SEP],key,fob,,,gar...",CERAMIC PIRATE CHEST MONEY BANK
11626,38283.0,"[CLS],funky,monkey,mug,[SEP],pink,blue,felt,cr...",ROUND SNACK BOXES SET OF 4 SKULLS


저장

In [ ]:
data_bert.to_csv("/content/drive/MyDrive/train_bert.csv", mode='w')
data_space.to_csv("/content/drive/MyDrive/train_space.csv", mode='w')